In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import tree

In [2]:
df = pd.read_csv("neo.csv",index_col=False)
df = df.drop(["id","name","orbiting_body","sentry_object"],axis=1)
df["hazardous"] = np.where(df["hazardous"] == False, 0, 1)
x = df.drop(["hazardous"],axis=1)
y= df["hazardous"]

In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42, train_size = 0.80)

## Decision Tree

In [5]:
model = tree.DecisionTreeClassifier()

In [6]:
model = model.fit(X_train,y_train)

In [11]:
yhat = model.predict(X_test)

In [12]:
from sklearn.metrics import f1_score
f1_score(y_test, yhat, average='micro')

0.8909070893879348

In [13]:
from sklearn.metrics import classification_report
target_name=['false','true']
print(classification_report(y_test, yhat, target_names=target_name))

              precision    recall  f1-score   support

       false       0.94      0.93      0.94     16439
        true       0.43      0.48      0.45      1729

    accuracy                           0.89     18168
   macro avg       0.69      0.71      0.70     18168
weighted avg       0.90      0.89      0.89     18168



## Random Forest classification

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
model = RandomForestClassifier(n_estimators=1000,
                               criterion='gini', 
                               max_depth=None,
                               min_samples_split=2, 
                               min_samples_leaf=1,
                               min_weight_fraction_leaf=0.0, 
                               max_features=None, 
                               max_leaf_nodes=None,
                               min_impurity_decrease=0.0,
                               bootstrap=True, 
                               oob_score=False,
                               n_jobs=-1,
                               random_state=3,
                               verbose=0, 
                               warm_start=False,
                               class_weight=None,
                               ccp_alpha=0.0,
                               max_samples=0.1)

In [41]:
model= model.fit(X_train, y_train)

In [42]:
yhat = model.predict(X_test)

In [43]:
f1_score(y_test, yhat, average='micro')

0.914795244385733

In [44]:
target_name=['false','true']
print(classification_report(y_test, yhat, target_names=target_name))

              precision    recall  f1-score   support

       false       0.92      0.99      0.95     16439
        true       0.71      0.18      0.28      1729

    accuracy                           0.91     18168
   macro avg       0.81      0.59      0.62     18168
weighted avg       0.90      0.91      0.89     18168



## XGBoost 

In [83]:
from xgboost import XGBClassifier as xgbc

In [84]:
model =  xgbc(n_estimators =1000, n_jobs=-1,eval_metric="auc")

In [85]:
evaluation = [( X_train, y_train), ( X_test, y_test)]
model=model.fit(X_train, y_train,
            eval_set=evaluation,
            early_stopping_rounds=10,verbose=False)

C:\Users\abhin\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [86]:
yhat = model.predict(X_test)

In [87]:
f1_score(y_test, yhat, average='micro')

0.9185380889476001

In [88]:
target_name=['false','true']
print(classification_report(y_test, yhat, target_names=target_name))

              precision    recall  f1-score   support

       false       0.93      0.99      0.96     16439
        true       0.71      0.24      0.36      1729

    accuracy                           0.92     18168
   macro avg       0.82      0.62      0.66     18168
weighted avg       0.91      0.92      0.90     18168

